[View in Colaboratory](https://colab.research.google.com/github/Joel-hanson/learntf/blob/master/assignment_01_colab.ipynb)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import tensorflow as tf
from datetime import datetime

In [5]:

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


# In[4]:


def weight_variables(shape):
    initial =  tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial =  tf.constant(0.1,shape=shape)
    return tf.Variable(initial)


# In[5]:


def conv_layer(x,W):
    return tf.nn.conv2d(x,W,[1,1,1,1],padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


# In[6]:


x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x,[-1,28,28,1])

weight_1 = weight_variables([5,5,1,32])
bias_1 = bias_variable([32])
conv_1 = tf.nn.relu(conv_layer(x_image,weight_1)+bias_1)
poll_1 = max_pool(conv_1)


# In[7]:


weight_2 = weight_variables([5,5,32,64])
bias_2 = bias_variable([64])
conv_2 = tf.nn.relu(conv_layer(poll_1,weight_2)+bias_2)
poll_2 = max_pool(conv_2)


# In[8]:


weight_fl = weight_variables([7*7*64,1024])
bias_fl = bias_variable([1024])
poll_fl = tf.reshape(poll_2,[-1, 7*7*64])
final_layer = tf.nn.relu(tf.matmul(poll_fl,weight_fl)+bias_fl)


# In[9]:


keep_pro = tf.placeholder(tf.float32)
final_layer_dropout = tf.nn.dropout(final_layer,keep_pro)


# In[10]:


#read layer
W_final = weight_variables([1024,10])
b_final = bias_variable([10])

y_conv = tf.matmul(final_layer_dropout,W_final)+b_final


# In[11]:


startTime = datetime.now()
with tf.device('/gpu:0'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(10000):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x:batch[0],y_:batch[1],keep_pro:1.0
                })
                print('step %d training accuracy %g' % (i,train_accuracy))
            train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_pro:0.5})
        print("test accuracy %g" % accuracy.eval(feed_dict={
            x:mnist.test.images,y_:mnist.test.labels,keep_pro:1.0
        }))
print("Time taken:", datetime.now() - startTime)



Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step 0 training accuracy 0.08
step 100 training accuracy 0.76
step 200 training accuracy 0.94
step 300 training accuracy 0.96
step 400 training accuracy 0.96
step 500 training accuracy 0.98
step 600 training accuracy 0.94
step 700 training accuracy 0.96
step 800 training accuracy 0.98
step 900 training accuracy 0.94
step 1000 training accuracy 0.9
step 1100 training accuracy 0.98
step 1200 training accuracy 0.98
step 1300 training accuracy 1
step 1400 training accuracy 0.98
step 1500 training accuracy 0.98
step 1600 training accuracy 1
step 1700 training accuracy 0.98
step 1800 training accuracy 0.98
step 1

step 6200 training accuracy 1
step 6300 training accuracy 1
step 6400 training accuracy 0.94
step 6500 training accuracy 1
step 6600 training accuracy 1
step 6700 training accuracy 1
step 6800 training accuracy 0.96
step 6900 training accuracy 0.98
step 7000 training accuracy 1
step 7100 training accuracy 1
step 7200 training accuracy 1
step 7300 training accuracy 1
step 7400 training accuracy 1
step 7500 training accuracy 1
step 7600 training accuracy 1
step 7700 training accuracy 1
step 7800 training accuracy 1
step 7900 training accuracy 1
step 8000 training accuracy 0.98
step 8100 training accuracy 1
step 8200 training accuracy 1
step 8300 training accuracy 1
step 8400 training accuracy 1
step 8500 training accuracy 1
step 8600 training accuracy 1
step 8700 training accuracy 1
step 8800 training accuracy 1
step 8900 training accuracy 1
step 9000 training accuracy 1
step 9100 training accuracy 1
step 9200 training accuracy 1
step 9300 training accuracy 0.98
step 9400 training accura

In [7]:
startTime = datetime.now()
with tf.device('/cpu:0'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(10000):
            batch = mnist.train.next_batch(50)
            if i % 100 == 0:
                train_accuracy = accuracy.eval(feed_dict={
                    x:batch[0],y_:batch[1],keep_pro:1.0
                })
                print('step %d training accuracy %g' % (i,train_accuracy))
            train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_pro:0.5})
        print("test accuracy %g" % accuracy.eval(feed_dict={
            x:mnist.test.images,y_:mnist.test.labels,keep_pro:1.0
        }))
print("Time taken:", datetime.now() - startTime)



step 0 training accuracy 0.06
step 100 training accuracy 0.64
step 200 training accuracy 0.86
step 300 training accuracy 0.9
step 400 training accuracy 0.94
step 500 training accuracy 0.98
step 600 training accuracy 0.92
step 700 training accuracy 0.9
step 800 training accuracy 0.96
step 900 training accuracy 0.96
step 1000 training accuracy 0.94
step 1100 training accuracy 0.98
step 1200 training accuracy 0.96
step 1300 training accuracy 0.98
step 1400 training accuracy 0.96
step 1500 training accuracy 0.94
step 1600 training accuracy 0.96
step 1700 training accuracy 1
step 1800 training accuracy 0.96
step 1900 training accuracy 0.94
step 2000 training accuracy 0.96
step 2100 training accuracy 0.98
step 2200 training accuracy 1
step 2300 training accuracy 0.94
step 2400 training accuracy 0.96
step 2500 training accuracy 1
step 2600 training accuracy 1
step 2700 training accuracy 0.98
step 2800 training accuracy 1
step 2900 training accuracy 0.98
step 3000 training accuracy 1
step 3100

step 7800 training accuracy 1
step 7900 training accuracy 0.98
step 8000 training accuracy 1
step 8100 training accuracy 1
step 8200 training accuracy 0.98
step 8300 training accuracy 1
step 8400 training accuracy 0.98
step 8500 training accuracy 1
step 8600 training accuracy 1
step 8700 training accuracy 0.98
step 8800 training accuracy 0.98
step 8900 training accuracy 0.98
step 9000 training accuracy 0.98
step 9100 training accuracy 0.96
step 9200 training accuracy 1
step 9300 training accuracy 1
step 9400 training accuracy 1
step 9500 training accuracy 1
step 9600 training accuracy 1
step 9700 training accuracy 1
step 9800 training accuracy 0.96
step 9900 training accuracy 1
test accuracy 0.9907
Time taken: 0:13:34.846950
